In [32]:
import pandas as pd
import numpy as np
import warnings
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.svm import LinearSVC
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
#from xgboost import XGBClassifier
from sklearn.metrics import confusion_matrix
from sklearn.metrics import roc_auc_score
from sklearn.metrics import make_scorer, accuracy_score, precision_score, recall_score, f1_score
from sklearn.metrics import classification_report
from sklearn.model_selection import cross_val_score, cross_validate
import seaborn as sns
import matplotlib.pyplot as plt

myData=pd.read_csv('online_data_set.csv')
myData.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,SpecialDay,Month,OperatingSystems,Browser,Region,TrafficType,VisitorType,Weekend,Revenue
0,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,1,1,1,1,Returning_Visitor,False,False
1,0,0.0,0,0.0,2,64.000000,0.00,0.10,0.0,0.0,Feb,2,2,1,2,Returning_Visitor,False,False
2,0,0.0,0,0.0,1,0.000000,0.20,0.20,0.0,0.0,Feb,4,1,9,3,Returning_Visitor,False,False
3,0,0.0,0,0.0,2,2.666667,0.05,0.14,0.0,0.0,Feb,3,2,2,4,Returning_Visitor,False,False
4,0,0.0,0,0.0,10,627.500000,0.02,0.05,0.0,0.0,Feb,3,3,1,4,Returning_Visitor,True,False


In [33]:
#One hot encoding of categorical variables

#Create list with features to be dummified cols.
nonum_feats_names = ['Month','OperatingSystems','Browser','Region','VisitorType','Weekend']


#Boolean to dummify logic
#Weekend_map={False:0,True:1}
#pd.get_dummies(myData['Weekend'].map(Weekend_map).astype('category')


dataModel = pd.concat([myData[['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']],
                       pd.get_dummies(myData[nonum_feats_names].astype('category')),myData['Revenue']],axis=1)
                      
dataModel.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12330 entries, 0 to 12329
Data columns (total 57 columns):
 #   Column                         Non-Null Count  Dtype  
---  ------                         --------------  -----  
 0   Administrative                 12330 non-null  int64  
 1   Administrative_Duration        12330 non-null  float64
 2   Informational                  12330 non-null  int64  
 3   Informational_Duration         12330 non-null  float64
 4   ProductRelated                 12330 non-null  int64  
 5   ProductRelated_Duration        12330 non-null  float64
 6   BounceRates                    12330 non-null  float64
 7   ExitRates                      12330 non-null  float64
 8   PageValues                     12330 non-null  float64
 9   TrafficType                    12330 non-null  int64  
 10  SpecialDay                     12330 non-null  float64
 11  Month_Aug                      12330 non-null  uint8  
 12  Month_Dec                      12330 non-null 

In [34]:
#Label Encoding of revenue

from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
myData['Revenue'] = le.fit_transform(myData['Revenue'])
myData['Revenue'].value_counts()

myData['Revenue'].head()

0    0
1    0
2    0
3    0
4    0
Name: Revenue, dtype: int64

In [35]:
# getting dependent and independent variables

x = dataModel
# removing the target column revenue from x
x = x.drop(['Revenue'], axis = 1)

y = myData['Revenue']

# checking the shapes
print("Shape of x:", x.shape)
print("Shape of y:", y.shape)

Shape of x: (12330, 56)
Shape of y: (12330,)


In [36]:
# splitting the data

from sklearn.model_selection import train_test_split

x_baseTrain, x_baseTest, y_baseTrain, y_baseTest = train_test_split(x, y, test_size = 0.3, random_state = 42)

# checking the shapes

print("Shape of x_train :", x_baseTrain.shape)
print("Shape of y_train :", y_baseTrain.shape)
print("Shape of x_test :", x_baseTest.shape)
print("Shape of y_test :", y_baseTest.shape)

Shape of x_train : (8631, 56)
Shape of y_train : (8631,)
Shape of x_test : (3699, 56)
Shape of y_test : (3699,)


In [37]:
#SCALING WITH STANDARD Z SCORE SCALER
#Scaling the data first fitting it and transforming the training set
#to later apply the fit to transform the test set.
from sklearn.preprocessing import StandardScaler

num_cols_names = ['Administrative', 'Administrative_Duration', 'Informational','Informational_Duration', 
                              'ProductRelated','ProductRelated_Duration','BounceRates','ExitRates','PageValues',
                               'TrafficType','SpecialDay']

#Instantiate Satandard Scaler
scaler = StandardScaler()
#Fit transform the numerical features in the training dataset to a new dataframe
scaled_numfeats_train = pd.DataFrame(scaler.fit_transform(x_baseTrain[num_cols_names]), 
                                     columns=num_cols_names, index= x_baseTrain.index)
#Integrate scaled values to the training set
for col in num_cols_names:
    x_baseTrain[col] = scaled_numfeats_train[col]
    
    
#Transform the numerical features inthe training dataset to a new dataframe
scaled_numfeats_test = pd.DataFrame(scaler.transform(x_baseTest[num_cols_names]),
                                    columns=num_cols_names, index= x_baseTest.index)
#Integrate scaled values to the test set
for col in num_cols_names:
    x_baseTest[col] = scaled_numfeats_test[col]

In [38]:
pd.set_option('display.max_columns', 60)
x_baseTest.head()

,Administrative,Administrative_Duration,Informational,Informational_Duration,ProductRelated,ProductRelated_Duration,BounceRates,ExitRates,PageValues,TrafficType,SpecialDay,Month_Aug,Month_Dec,Month_Feb,Month_Jul,Month_June,Month_Mar,Month_May,Month_Nov,Month_Oct,Month_Sep,OperatingSystems_1,OperatingSystems_2,OperatingSystems_3,OperatingSystems_4,OperatingSystems_5,OperatingSystems_6,OperatingSystems_7,OperatingSystems_8,Browser_1,Browser_2,Browser_3,Browser_4,Browser_5,Browser_6,Browser_7,Browser_8,Browser_9,Browser_10,Browser_11,Browser_12,Browser_13,Region_1,Region_2,Region_3,Region_4,Region_5,Region_6,Region_7,Region_8,Region_9,VisitorType_New_Visitor,VisitorType_Other,VisitorType_Returning_Visitor,Weekend_False,Weekend_True
8916,0.211976,0.358157,-0.397205,-0.252976,0.367362,-0.078457,-0.362556,-0.616331,-0.314838,1.721701,-0.3047,0,0,0,0,0,0,0,1,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,1,1,0
772,1.123567,2.061937,1.164298,1.483545,1.156066,0.715483,-0.408041,-0.786962,-0.204095,-0.515315,-0.3047,0,0,0,0,0,1,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,1,0
12250,-0.395752,-0.227552,-0.397205,-0.252976,2.125046,1.703310,-0.439985,-0.620964,-0.131644,-0.515315,-0.3047,0,0,0,0,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,1,1,0
7793,-0.091888,0.349491,-0.397205,-0.252976,-0.488945,-0.322165,-0.278241,-0.336149,1.631839,-0.018200,-0.3047,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,1,1,0
6601,4.769931,3.048464,4.287304,5.156742,3.071491,2.052468,-0.314238,-0.606169,0.223989,-0.763872,-0.3047,1,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0,0,0,0,1,0,1


In [39]:
# MODELLING ____RANDOM FOREST

model = RandomForestClassifier()  #can try estimators=200,mex depth=30
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 1.0
Testing Accuracy : 0.8953771289537713
ROC AUC Score : 0.7365582029727774
[[3021  103]
 [ 284  291]]
              precision    recall  f1-score   support

           0       0.91      0.97      0.94      3124
           1       0.74      0.51      0.60       575

    accuracy                           0.90      3699
   macro avg       0.83      0.74      0.77      3699
weighted avg       0.89      0.90      0.89      3699

{'fit_time': array([2.24399972, 2.18728352, 2.21841431, 1.65600538, 2.07098055,
       2.22143269, 2.19146562, 2.18895745, 2.15019441, 2.22050452]), 'score_time': array([0.07385492, 0.08463979, 0.0632391 , 0.06527996, 0.08648396,
       0.08270717, 0.07853651, 0.06285405, 0.07872224, 0.06306291]), 'test_accuracy': array([0.90046296, 0.88760139, 0.89918888, 0.91772885, 0.90961761,
       0.91541136, 0.89339513, 0.90730012, 0.89918888, 0.89455388]), 'test_precision': array([0.77272727, 0.7       , 0.76136364, 0.84444444, 0.78947368,
       0.812

In [40]:
# MODELLING ____KNN

model = KNeighborsClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))


# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.9027922604564941
Testing Accuracy : 0.8705055420383887
ROC AUC Score : 0.6650723709848021
[[3009  115]
 [ 364  211]]
              precision    recall  f1-score   support

           0       0.89      0.96      0.93      3124
           1       0.65      0.37      0.47       575

    accuracy                           0.87      3699
   macro avg       0.77      0.67      0.70      3699
weighted avg       0.85      0.87      0.86      3699

{'fit_time': array([0.01451612, 0.01394153, 0.02296615, 0.01562214, 0.00984025,
       0.01563311, 0.02156591, 0.01168656, 0.01566792, 0.01156688]), 'score_time': array([0.38634467, 0.42531204, 0.39876962, 0.23534894, 0.23965335,
       0.24143076, 0.2228415 , 0.2394619 , 0.23395061, 0.25516629]), 'test_accuracy': array([0.87037037, 0.86790267, 0.87485516, 0.88876014, 0.87717265,
       0.89687138, 0.86558517, 0.8806489 , 0.8806489 , 0.86790267]), 'test_precision': array([0.65714286, 0.63768116, 0.68115942, 0.74025974, 0.6626506

In [41]:
# MODELLING ____Ada Boost Classifier

model = AdaBoostClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.8966516046808017
Testing Accuracy : 0.8872668288726683
ROC AUC Score : 0.7487850025051495
[[2967  157]
 [ 260  315]]
              precision    recall  f1-score   support

           0       0.92      0.95      0.93      3124
           1       0.67      0.55      0.60       575

    accuracy                           0.89      3699
   macro avg       0.79      0.75      0.77      3699
weighted avg       0.88      0.89      0.88      3699

{'fit_time': array([1.57184482, 1.40086508, 1.41250038, 1.42785525, 1.39590573,
       1.45195794, 1.42988038, 1.22487736, 1.05521011, 1.17097592]), 'score_time': array([0.05644584, 0.04680538, 0.05286694, 0.05821133, 0.05001092,
       0.04892945, 0.04743147, 0.04550838, 0.04280686, 0.04786539]), 'test_accuracy': array([0.90046296, 0.87485516, 0.89339513, 0.90498262, 0.91772885,
       0.89455388, 0.88180765, 0.88528389, 0.89571263, 0.89223638]), 'test_precision': array([0.73076923, 0.62886598, 0.68807339, 0.73394495, 0.7627118

In [48]:
conda install -c conda-forge xgboost

Solving environment: ...working... done

## Package Plan ##

  environment location: C:\ProgramData\Anaconda3

  added / updated specs:
    - xgboost


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    _py-xgboost-mutex-2.0      |            cpu_0          11 KB  conda-forge
    conda-4.14.0               |   py39hcbf5309_0         1.0 MB  conda-forge
    libxgboost-1.7.3           |       hd77b12b_0         1.5 MB
    py-xgboost-1.7.3           |   py39haa95532_0         197 KB
    python_abi-3.9             |           2_cp39           4 KB  conda-forge
    xgboost-1.7.3              |   py39haa95532_0          12 KB
    ------------------------------------------------------------
                                           Total:         2.8 MB

The following NEW packages will be INSTALLED:


Note: you may need to restart the kernel to use updated packages.  _py-xgboost-mutex  conda-forg



==> WARNING: A newer version of conda exists. <==
  current version: 4.12.0
  latest version: 23.5.0

Please update conda by running

    $ conda update -n base -c defaults conda



EnvironmentNotWritableError: The current user does not have write permissions to the target environment.
  environment location: C:\ProgramData\Anaconda3




py-xgboost-1.7.3     | 197 KB    |            |   0% 
py-xgboost-1.7.3     | 197 KB    | 8          |   8% 
py-xgboost-1.7.3     | 197 KB    | #####6     |  57% 
py-xgboost-1.7.3     | 197 KB    | ########## | 100% 
py-xgboost-1.7.3     | 197 KB    | ########## | 100% 

_py-xgboost-mutex-2. | 11 KB     |            |   0% 
_py-xgboost-mutex-2. | 11 KB     | ########## | 100% 
_py-xgboost-mutex-2. | 11 KB     | ########## | 100% 

libxgboost-1.7.3     | 1.5 MB    |            |   0% 
libxgboost-1.7.3     | 1.5 MB    | 1          |   1% 
libxgboost-1.7.3     | 1.5 MB    | #1         |  11% 
libxgboost-1.7.3     | 1.5 MB    | #9         |  20% 
libxgboost-1.7.3     | 1.5 MB    | ###2       |  33% 
libxgboost-1.7.3     | 1.5 MB    | ####3      |  43% 
libxgboost-1.7.3     | 1.5 MB    | #####2     |  52% 
libxgboost-1.7.3     | 1.5 MB    | ######3    |  64% 
libxgboost-1.7.3     | 1.5 MB    | #######2   |  73% 
libxgboost-1.7.3     | 1.5 MB    | ########1  |  81% 
libxgboost-1.7.3     | 1.5

sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1d79f6c71c0>
C:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedWriter name=4>
  del self._target, self._args, self._kwargs
C:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedReader name=6>
  del self._target, self._args, self._kwargs
C:\ProgramData\Anaconda3\lib\threading.py:914: ResourceWarning: unclosed file <_io.BufferedReader name=7>
  del self._target, self._args, self._kwargs
sys:1: ResourceWarning: unclosed socket <zmq.Socket(zmq.PUSH) at 0x1d79f6d0880>


In [49]:
import xgboost

ModuleNotFoundError: No module named 'xgboost'

In [45]:
# MODELLING ____XGBOOST classifier

model = 
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.9213300892133008
Testing Accuracy : 0.8951067856177345
ROC AUC Score : 0.7740023938094973
[[2967  157]
 [ 231  344]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3124
           1       0.69      0.60      0.64       575

    accuracy                           0.90      3699
   macro avg       0.81      0.77      0.79      3699
weighted avg       0.89      0.90      0.89      3699

{'fit_time': array([3.83380723, 4.68196988, 4.37886143, 4.19091916, 3.62909198,
       4.85135508, 4.59028888, 3.86410379, 4.52626896, 4.72028112]), 'score_time': array([0.02031803, 0.01853871, 0.01675534, 0.        , 0.02782559,
       0.01656508, 0.02311587, 0.01562119, 0.01563597, 0.0156467 ]), 'test_accuracy': array([0.90625   , 0.8829664 , 0.90614137, 0.91772885, 0.91772885,
       0.90730012, 0.89571263, 0.90730012, 0.90266512, 0.90034762]), 'test_precision': array([0.73043478, 0.64545455, 0.73214286, 0.78703704, 0.7719298

In [46]:
# MODELLING ____gradient boosting Classifier

model = GradientBoostingClassifier()
model.fit(x_baseTrain, y_baseTrain)

y_basePred = model.predict(x_baseTest)

# evaluating the model
print("Training Accuracy :", model.score(x_baseTrain, y_baseTrain))
print("Testing Accuracy :", model.score(x_baseTest, y_baseTest))
print("ROC AUC Score :", roc_auc_score(y_baseTest, y_basePred))

# confusion matrix
cm = confusion_matrix(y_baseTest, y_basePred)
print(cm)

# classification report
cr = classification_report(y_baseTest, y_basePred)
print(cr)


# cross validation
scoring = {'accuracy' : make_scorer(accuracy_score), 
           'precision' : make_scorer(precision_score),
           'recall' : make_scorer(recall_score), 
           'f1_score' : make_scorer(f1_score)}
cvs = cross_validate(estimator = model, X = x_baseTrain, y = y_baseTrain, cv = 10,scoring=scoring)
print(cvs)
print("Mean Accuracy :", np.mean(cvs['test_accuracy']))
print("Mean Standard Deviation :", np.std(cvs['test_accuracy']))
print("Mean precision score :", np.mean(cvs['test_precision']))
print("Mean Standard Deviation precision score :", np.std(cvs['test_precision']))
print("Mean recall score :", np.mean(cvs['test_recall']))
print("Mean Standard Deviation recall score :", np.std(cvs['test_recall']))
print("Mean f1 score :", np.mean(cvs['test_f1_score']))
print("Mean Standard Deviation f1 score :", np.std(cvs['test_f1_score']))

Training Accuracy : 0.9213300892133008
Testing Accuracy : 0.8953771289537713
ROC AUC Score : 0.7748719590268887
[[2967  157]
 [ 230  345]]
              precision    recall  f1-score   support

           0       0.93      0.95      0.94      3124
           1       0.69      0.60      0.64       575

    accuracy                           0.90      3699
   macro avg       0.81      0.77      0.79      3699
weighted avg       0.89      0.90      0.89      3699

{'fit_time': array([4.57512736, 4.43915343, 3.79475665, 4.36160541, 4.45735669,
       4.38460207, 3.49002695, 4.58151746, 4.61807084, 3.91089821]), 'score_time': array([0.01921391, 0.03139544, 0.01562548, 0.02165866, 0.01209259,
       0.0160346 , 0.0065062 , 0.00819707, 0.02244663, 0.01346993]), 'test_accuracy': array([0.90625   , 0.88180765, 0.90614137, 0.91772885, 0.91772885,
       0.90730012, 0.89571263, 0.90730012, 0.90266512, 0.90034762]), 'test_precision': array([0.73043478, 0.64220183, 0.73214286, 0.78703704, 0.7719298